In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd  /content/drive/'My Drive'/NYU_Spring_2021/CCM/code/vonenet-tensorflow

Mounted at /content/drive
/content/drive/My Drive/NYU_Spring_2021/CCM/code/vonenet-tensorflow


In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import gzip
import os
import re
import sys
import tarfile

from six.moves import urllib
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt

In [3]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from vonenet.layers import VOneNetLayer

In [4]:
def read_tfrecord(examples, labeled=True):
    tfrecord_format = (
        {
            "image_raw": tf.io.FixedLenFeature([], tf.string),
            "label": tf.io.FixedLenFeature([], tf.int64),
        }
    )
    example= tf.io.parse_single_example(examples, tfrecord_format)
    image = tf.io.decode_raw(example["image_raw"], tf.uint8)
    image = tf.cast(image, tf.float32) * (1. / 255)
    image.set_shape(np.prod([5, 60, 60]))
    image = tf.reshape(image, [5, 60, 60, 1])
    image = image - 0.5
    label = tf.cast(example["label"], tf.int32)
    label = tf.one_hot(label, 5)
    # label = tf.keras.utils.to_categorical(y=label, num_classes=5, dtype=tf.int32)
    return image, label

In [5]:
def load_dataset(filename, labeled=True):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False  # disable order, increase speed
    dataset = tf.data.TFRecordDataset(
        filename
    )  # automatically interleaves reads from multiple files
    dataset = dataset.with_options(
        ignore_order
    )  # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(
        partial(read_tfrecord, labeled=labeled), num_parallel_calls=AUTOTUNE
    )
    # returns a dataset of (image, label) pairs if labeled=True or just images if labeled=False
    return dataset

In [6]:
def get_dataset(filename, labeled=True):
    dataset = load_dataset(filename, labeled=labeled)
    dataset = dataset.shuffle(2048)
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    dataset = dataset.batch(128)
    return dataset

In [7]:
from functools import partial
AUTOTUNE = tf.data.experimental.AUTOTUNE


train_filename = '../eccentricity/experiments/demo-experiment/datasets/gen_fl_0/data/train_dh=120-flanker_h=120-train_ecc=0.0-ns=5-rs=False-p=even-ftype=0-te=0.0-fe=0.0-c=inf-fd=MNIST-cnorm=None-place=None.tfrecords'
val_filename = '../eccentricity/experiments/demo-experiment/datasets/gen_fl_0/data/validation_dh=120-flanker_h=120-train_ecc=0.0-ns=5-rs=False-p=even-ftype=0-te=0.0-fe=0.0-c=inf-fd=MNIST-cnorm=None-place=None.tfrecords'
test_filename = '../eccentricity/experiments/demo-experiment/datasets/gen_fl_0/data/test_dh=120-flanker_h=120-train_ecc=0.0-ns=5-rs=False-p=even-ftype=0-te=0.0-fe=0.0-c=inf-fd=MNIST-cnorm=None-place=None.tfrecords'

train_dataset = get_dataset(train_filename)
val_dataset = get_dataset(val_filename)
test_dataset = get_dataset(test_filename)


In [8]:
input_shape = (128,5,60,60,1)
num_classes = 5


In [9]:
image, label = next(iter(train_dataset))
print(image.shape, label.shape)

(128, 5, 60, 60, 1) (128, 5)


In [30]:
res = VOneNetLayer(shape=input_shape[2:], ksize=5, stride=2, simple_channels=32, complex_channels=32)(image)

Neuronal distributions gabor parameters


/content/drive/My Drive/NYU_Spring_2021/CCM/code/vonenet-tensorflow/vonenet/utils.py:86: RuntimeWarning: invalid value encountered in true_divide
  ny_dist_marg = n_joint_dist / n_joint_dist.sum(axis=1, keepdims=True)


In [46]:
res.shape

TensorShape([128, 5, 30, 30, 64])

In [47]:
x = layers.Conv2D(filters=32, kernel_size=3, activation='relu')(res)

In [48]:
x.shape

TensorShape([128, 5, 28, 28, 32])

In [49]:
x = layers.MaxPooling3D(pool_size=(1,3,3), strides=(1,2,2))(x)
x.shape

TensorShape([128, 5, 13, 13, 32])

In [50]:
x = layers.Conv2D(filters=32, kernel_size=3, activation='relu')(x)

In [51]:
x.shape

TensorShape([128, 5, 11, 11, 32])

In [52]:
y = layers.MaxPooling3D(pool_size=(1,3,3), strides=(1,2,2))(x)

In [53]:
y.shape

TensorShape([128, 5, 5, 5, 32])

In [55]:
y = layers.Flatten()(y)
y.shape

TensorShape([128, 4000])

In [10]:
model = keras.Sequential(
    [   layers.Input(shape=input_shape[1:]), #[128,5,60,60,1]
        VOneNetLayer(shape=input_shape[2:], ksize=5, stride=2, simple_channels=32, complex_channels=32), #[128,5,30,30,64]
        layers.Conv2D(filters=32, kernel_size=3, activation='relu'), #[128, 5, 28, 28, 32]
        layers.MaxPooling3D(pool_size=(1,3,3), strides=(1,2,2)), # [128, 5, 13, 13, 32]
        layers.Conv2D(filters=32, kernel_size=3, activation='relu'), # [128, 5, 11, 11, 32]
        layers.MaxPooling3D(pool_size=(1,3,3), strides=(1,2,2)), # [128, 5, 5, 5, 32]
        layers.Flatten(), # [128, 4000]
        layers.Dense(num_classes, activation="softmax") # [128, 5]
    ]
)

Neuronal distributions gabor parameters


/content/drive/My Drive/NYU_Spring_2021/CCM/code/vonenet-tensorflow/vonenet/utils.py:86: RuntimeWarning: invalid value encountered in true_divide
  ny_dist_marg = n_joint_dist / n_joint_dist.sum(axis=1, keepdims=True)


In [11]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
v_one_net_layer (VOneNetLaye (None, 5, 30, 30, 64)     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 5, 28, 28, 32)     18464     
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 5, 13, 13, 32)     0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 5, 11, 11, 32)     9248      
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 5, 5, 5, 32)       0         
_________________________________________________________________
flatten (Flatten)            (None, 4000)              0         
_________________________________________________________________
dense (Dense)                (None, 5)                 2

In [12]:
opt = keras.optimizers.Adagrad(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])


In [13]:
epochs = 20
history = model.fit(
    train_dataset,
    epochs=epochs,
    validation_data=val_dataset,
)

Epoch 1/20
212/212 [==============================] - 27s 53ms/step - loss: 4.3967 - accuracy: 0.2110 - val_loss: 0.5508 - val_accuracy: 0.8247
Epoch 2/20
212/212 [==============================] - 8s 36ms/step - loss: 0.3187 - accuracy: 0.8985 - val_loss: 0.2117 - val_accuracy: 0.9294
Epoch 3/20
212/212 [==============================] - 8s 36ms/step - loss: 0.1080 - accuracy: 0.9652 - val_loss: 0.1853 - val_accuracy: 0.9355
Epoch 4/20
212/212 [==============================] - 8s 36ms/step - loss: 0.0878 - accuracy: 0.9723 - val_loss: 0.0613 - val_accuracy: 0.9797
Epoch 5/20
212/212 [==============================] - 8s 36ms/step - loss: 0.0692 - accuracy: 0.9791 - val_loss: 0.0633 - val_accuracy: 0.9781
Epoch 6/20
212/212 [==============================] - 8s 36ms/step - loss: 0.0674 - accuracy: 0.9785 - val_loss: 0.0528 - val_accuracy: 0.9809
Epoch 7/20
212/212 [==============================] - 8s 36ms/step - loss: 0.0561 - accuracy: 0.9816 - val_loss: 0.0630 - val_accuracy: 0.980

In [14]:
# No flanker
score = model.evaluate(test_dataset, verbose=0)

In [15]:
score

[0.03756945952773094, 0.9886317253112793]

In [18]:
# xa, e=240px
test_filename = '../eccentricity/experiments/demo-experiment/datasets/gen_fl_1_xa_240/data/test_dh=120-flanker_h=120-train_ecc=0.0-ns=5-rs=False-p=even-ftype=1-te=120.0-fe=240.0-c=inf-fd=MNIST-cnorm=None-place=None.tfrecords'
test_dataset = get_dataset(test_filename)

In [19]:
score = model.evaluate(test_dataset, verbose=0)
print(score)

[2.5385215282440186, 0.19853836297988892]


In [20]:
# xa, e = 480px
test_filename = '../eccentricity/experiments/demo-experiment/datasets/gen_fl_1_xa_480/data/test_dh=120-flanker_h=120-train_ecc=0.0-ns=5-rs=False-p=even-ftype=1-te=0.0-fe=480.0-c=inf-fd=MNIST-cnorm=None-place=None.tfrecords'
test_dataset = get_dataset(test_filename)
score = model.evaluate(test_dataset, verbose=0)
print(score)

[0.04025980085134506, 0.9874137043952942]


In [21]:
# xax, e = 480px
test_filename = '../eccentricity/experiments/demo-experiment/datasets/gen_fl_2_xax_480/data/test_dh=120-flanker_h=120-train_ecc=0.0-ns=5-rs=False-p=even-ftype=2-te=0.0-fe=480.0-c=inf-fd=MNIST-cnorm=None-place=None.tfrecords'
test_dataset = get_dataset(test_filename)
score = model.evaluate(test_dataset, verbose=0)
print(score)

[0.03828917443752289, 0.9878197312355042]


In [22]:
# xa, e=360px
test_filename = '../eccentricity/experiments/demo-experiment/datasets/gen_fl_1_120/data/test_dh=120-flanker_h=120-train_ecc=0.0-ns=5-rs=False-p=even-ftype=1-te=120.0-fe=360.0-c=inf-fd=MNIST-cnorm=None-place=None.tfrecords'
test_dataset = get_dataset(test_filename)
score = model.evaluate(test_dataset, verbose=0)
print(score)

[2.489708423614502, 0.19529029726982117]


## Training Dataset 2 (xa, e=360px) ##

In [23]:
train_filename = '../eccentricity/experiments/demo-experiment/datasets/gen_fl_1_120/data/train_dh=120-flanker_h=120-train_ecc=0.0-ns=5-rs=False-p=even-ftype=1-te=120.0-fe=360.0-c=inf-fd=MNIST-cnorm=None-place=None.tfrecords'
val_filename = '../eccentricity/experiments/demo-experiment/datasets/gen_fl_1_120/data/validation_dh=120-flanker_h=120-train_ecc=0.0-ns=5-rs=False-p=even-ftype=1-te=120.0-fe=360.0-c=inf-fd=MNIST-cnorm=None-place=None.tfrecords'
test_filename = '../eccentricity/experiments/demo-experiment/datasets/gen_fl_1_120/data/test_dh=120-flanker_h=120-train_ecc=0.0-ns=5-rs=False-p=even-ftype=1-te=120.0-fe=360.0-c=inf-fd=MNIST-cnorm=None-place=None.tfrecords'

train_dataset_2 = get_dataset(train_filename)
val_dataset_2 = get_dataset(val_filename)
test_dataset_2 = get_dataset(test_filename)

In [24]:
model_2 = keras.Sequential(
    [   layers.Input(shape=input_shape[1:]), #[128,5,60,60,1]
        VOneNetLayer(shape=input_shape[2:], ksize=5, stride=2, simple_channels=32, complex_channels=32), #[128,5,30,30,64]
        layers.Conv2D(filters=32, kernel_size=3, activation='relu'), #[128, 5, 28, 28, 32]
        layers.MaxPooling3D(pool_size=(1,3,3), strides=(1,2,2)), # [128, 5, 13, 13, 32]
        layers.Conv2D(filters=32, kernel_size=3, activation='relu'), # [128, 5, 11, 11, 32]
        layers.MaxPooling3D(pool_size=(1,3,3), strides=(1,2,2)), # [128, 5, 5, 5, 32]
        layers.Flatten(), # [128, 4000]
        layers.Dense(num_classes, activation="softmax") # [128, 5]
    ]
)

Neuronal distributions gabor parameters


/content/drive/My Drive/NYU_Spring_2021/CCM/code/vonenet-tensorflow/vonenet/utils.py:86: RuntimeWarning: invalid value encountered in true_divide
  ny_dist_marg = n_joint_dist / n_joint_dist.sum(axis=1, keepdims=True)


In [25]:
model_2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
v_one_net_layer_1 (VOneNetLa (None, 5, 30, 30, 64)     0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 5, 28, 28, 32)     18464     
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 5, 13, 13, 32)     0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 5, 11, 11, 32)     9248      
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (None, 5, 5, 5, 32)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4000)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                

In [26]:
opt = keras.optimizers.Adagrad(learning_rate=0.01)
model_2.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])


In [27]:
epochs = 20
history_2 = model_2.fit(
    train_dataset_2,
    epochs=epochs,
    validation_data=val_dataset_2,
)

Epoch 1/20
212/212 [==============================] - 13s 53ms/step - loss: 3.2569 - accuracy: 0.3262 - val_loss: 0.3124 - val_accuracy: 0.8998
Epoch 2/20
212/212 [==============================] - 8s 36ms/step - loss: 0.2600 - accuracy: 0.9187 - val_loss: 0.1851 - val_accuracy: 0.9404
Epoch 3/20
212/212 [==============================] - 8s 36ms/step - loss: 0.1700 - accuracy: 0.9474 - val_loss: 0.1283 - val_accuracy: 0.9570
Epoch 4/20
212/212 [==============================] - 8s 35ms/step - loss: 0.1315 - accuracy: 0.9580 - val_loss: 0.1125 - val_accuracy: 0.9611
Epoch 5/20
212/212 [==============================] - 8s 36ms/step - loss: 0.1120 - accuracy: 0.9639 - val_loss: 0.0886 - val_accuracy: 0.9684
Epoch 6/20
212/212 [==============================] - 8s 36ms/step - loss: 0.0961 - accuracy: 0.9701 - val_loss: 0.0950 - val_accuracy: 0.9663
Epoch 7/20
212/212 [==============================] - 8s 36ms/step - loss: 0.0939 - accuracy: 0.9694 - val_loss: 0.0724 - val_accuracy: 0.978

In [28]:
score = model_2.evaluate(test_dataset_2, verbose=0)

In [29]:
print(score)

[0.05164167284965515, 0.9823386073112488]


In [30]:
# no flanker
test_filename = '../eccentricity/experiments/demo-experiment/datasets/gen_fl_0/data/test_dh=120-flanker_h=120-train_ecc=0.0-ns=5-rs=False-p=even-ftype=0-te=0.0-fe=0.0-c=inf-fd=MNIST-cnorm=None-place=None.tfrecords'
test_dataset = get_dataset(test_filename)
score = model_2.evaluate(test_dataset, verbose=0)
print(score)

[45.749046325683594, 0.26573285460472107]


In [34]:
# xa, e=240px
test_filename = '../eccentricity/experiments/demo-experiment/datasets/gen_fl_1_xa_240/data/test_dh=120-flanker_h=120-train_ecc=0.0-ns=5-rs=False-p=even-ftype=1-te=120.0-fe=240.0-c=inf-fd=MNIST-cnorm=None-place=None.tfrecords'
test_dataset = get_dataset(test_filename)
score = model_2.evaluate(test_dataset, verbose=0)
print(score)

[0.05567806959152222, 0.9801055788993835]


In [31]:
# xa, e = 480px
test_filename = '../eccentricity/experiments/demo-experiment/datasets/gen_fl_1_xa_480/data/test_dh=120-flanker_h=120-train_ecc=0.0-ns=5-rs=False-p=even-ftype=1-te=0.0-fe=480.0-c=inf-fd=MNIST-cnorm=None-place=None.tfrecords'
test_dataset = get_dataset(test_filename)
score = model_2.evaluate(test_dataset, verbose=0)
print(score)

[45.71614456176758, 0.2706049680709839]


In [33]:
#xax e=480
test_filename = '../eccentricity/experiments/demo-experiment/datasets/gen_fl_2_xax_480/data/test_dh=120-flanker_h=120-train_ecc=0.0-ns=5-rs=False-p=even-ftype=2-te=0.0-fe=480.0-c=inf-fd=MNIST-cnorm=None-place=None.tfrecords'
test_dataset = get_dataset(test_filename)
score = model_2.evaluate(test_dataset, verbose=0)
print(score)

[45.747432708740234, 0.2655298411846161]
